# Ansering Business Questions using SQL

## Creating Helper Functions

Before beginning the project, we should create a few helper functions that will help with querying later on in the project. 

In [2]:
import sqlite3, pandas as pd

def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)
    
def run_command(q):
    with sqlite3.connect('chinook.db') as conn:
        c = conn.cursor()
        return c.execute(q)

def show_tables():
    query = "SELECT name, type FROM sqlite_master WHERE type IN ('table','view');"
    return run_query(query)

show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## Selecting Albums to Purchase

Our record store has signed a deal with a new record label! The label is asking us to promote 3 albums of artists they have signed. We need to figure out which genres sell the best in the USA in order to make a better decision as to which albums we choose to promote.

In [3]:
q = """WITH best_genres AS
(
    SELECT 
        g.name genre_name,
        SUM(il.quantity) total_tracks_sold
    FROM invoice
    INNER JOIN invoice_line il ON il.invoice_id = invoice.invoice_id
    INNER JOIN track t ON t.track_id = il.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    WHERE billing_country = "USA"
    GROUP BY genre_name
    ORDER BY total_tracks_sold DESC
)

SELECT
    *, 
    CAST(total_tracks_sold AS float)/(SELECT SUM(total_tracks_sold) FROM best_genres) Percentage_of_Total
    FROM best_genres;"""

run_query(q)

,genre_name,total_tracks_sold,Percentage_of_Total
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


In [23]:
no_tracks_sold_usa = run_query(q)

Based on our queries, it appears that the the least popular genre of the 4 artist we are considering is Hip-Hop/Rap.

Therefore, the 3 artists whose albums we should purchase for the store are:
* Red Tone
* Metor and the Girls
* Slim Jim Bites

## Analyzing Employee Sales Performance

Let's now analyze the sales performance of the store's employees.

In [4]:
q = """WITH customer_total AS
    (
        SELECT 
            customer_id,
            SUM(total) total_spent
        FROM invoice  
        GROUP BY customer_id
    )
    
    SELECT
        e.first_name || " " || e.last_name sales_support_agent,
        ROUND(SUM(ct.total_spent),2) total_sales,
        e.title,
        e.reports_to,
        e.hire_date,
        e.birthdate
    FROM employee e
    INNER JOIN customer_total ct ON c.customer_id = ct.customer_id
    INNER JOIN customer c ON c.support_rep_id = e.employee_id
    GROUP BY employee_id;"""

run_query(q)


,sales_support_agent,total_sales,title,reports_to,hire_date,birthdate
0,Jane Peacock,1731.51,Sales Support Agent,2,2017-04-01 00:00:00,1973-08-29 00:00:00
1,Margaret Park,1584.00,Sales Support Agent,2,2017-05-03 00:00:00,1947-09-19 00:00:00
2,Steve Johnson,1393.92,Sales Support Agent,2,2017-10-17 00:00:00,1965-03-03 00:00:00


In [26]:
top_sales_staff = run_query(q)

From our analysis, we can see that Jane Peacock has the highest sales of 1731.51. This is followed by Margaret Park that made the total sales of 1584 and then Steve Johnson which made 1393.92 sales in total.

Margaret's sales is 147.51 less than Jane's while Steve's sales is 190.08 less than Margaret's.

As Jane has been working for the company the longest among the three, Jane has the most experience as a Sales Support Agent in Chinook. This could be a possible reason why Jane made the most sales. Margaret was hired a month later and then Steve was hired 5 months after Margaret. Again, the result table shows that the total sales made is relative to the order of the employee's hire date.

Another reason is that Jane is also the youngest among the three. She is probably able to relate to the customers better as the customers using Chinook might be of a similar age. Additionally, Jane might also be more knowledgeable since she might be using Chinook herself.

## Analyzing Sales by Country

Let us now query the database to analyze the sales data for customers from different countries outside of the USA.

In [5]:
q = """WITH 
    country_or_other AS
        (
            SELECT 
                CASE
                    WHEN COUNT(DISTINCT c.customer_id) = 1 THEN "Other"
                    ELSE c.country
                    END
                    AS country,
                COUNT(DISTINCT c.customer_id) total_customers,
                SUM(i.total) total_sales,
                COUNT(DISTINCT i.invoice_id) total_orders
            FROM customer c
            INNER JOIN invoice i ON i.customer_id = c.customer_id
            GROUP BY c.country            
        ),
    total_country_sales_customer AS
        (
        SELECT 
            country,
            SUM(total_customers) customers,
            SUM(total_sales) total_sales,
            SUM(total_orders) total_orders
        FROM country_or_other
        GROUP BY country
    )
                
SELECT 
    country,
    customers,
    total_sales,
    total_sales/customers avg_sales_per_customer,  
    total_sales/total_orders avg_order_value
FROM
    (
        SELECT
            *,
            CASE
                WHEN country = "Other" THEN 1
                ELSE 0
                END
                AS sort 
        FROM total_country_sales_customer tcsc
    )    
ORDER BY sort, total_sales DESC;"""

run_query(q)

,country,customers,total_sales,avg_sales_per_customer,avg_order_value
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,Czech Republic,2,273.24,136.620000,9.108000
6,United Kingdom,3,245.52,81.840000,8.768571
7,Portugal,2,185.13,92.565000,6.383793
8,India,2,183.15,91.575000,8.721429
9,Other,15,1094.94,72.996000,7.448571


Firstly, note that Czech Republic has an above average sales per customer of 136.62 compared to the rest of the countries whereby their values are lower than 100. Other than that, United Kingdom and India has a higher average order value compared to the rest of the countries (after Czech Republic). Therefore, there might be potential in these countries.

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Albums vs. Individual Tracks

Let's run queries to find out what percentage of purchases are individual tracks vs whole albums.

In [6]:
q = """WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;"""

run_query(q)

,album_purchase,number_of_invoices,percent
0,no,500,0.814332
1,yes,114,0.185668


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.

## Which artist is used in the most playlists?


In [7]:
q = """SELECT
    a.artist_id,
    ar.name artist_name,
    SUM(DISTINCT playlist_id) no_of_playlist
FROM track t
INNER JOIN playlist_track pt ON pt.track_id = t.track_id
INNER JOIN album a ON a.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = a.artist_id
GROUP BY a.artist_id 
ORDER BY no_of_playlist DESC
LIMIT 10;"""

run_query(q)

,artist_id,artist_name,no_of_playlist
0,226,Eugene Ormandy,68
1,208,English Concert & Trevor Pinnock,55
2,214,Academy of St. Martin in the Fields & Sir Nevi...,55
3,247,The King's Singers,53
4,248,Berliner Philharmoniker & Herbert Von Karajan,53
5,206,Alberto Turco & Nova Schola Gregoriana,41
6,207,"Richard Marlow & The Choir of Trinity College,...",41
7,211,Wilhelm Kempff,41
8,212,Yo-Yo Ma,41
9,213,Scholars Baroque Ensemble,41


## How many tracks have been purchased vs. not purchased?

In [8]:
q = """WITH
    tracks_purchased AS
    (
        SELECT 
            il.track_id
        FROM invoice i
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        GROUP BY il.track_id
    )
    
SELECT 
    COUNT(*) tracks_purchased,
    CAST(COUNT(*) as Float)/(SELECT COUNT(track_id) FROM track) percentage_purchased,
    (SELECT COUNT(track_id) FROM track) - COUNT(*) not_purchased,
    (SELECT COUNT(track_id) FROM track) total_tracks
FROM tracks_purchased;"""

run_query(q)

,tracks_purchased,percentage_purchased,not_purchased,total_tracks
0,1806,0.515558,1697,3503


## Is the range of tracks in the store reflective of their sales popularity?

In [9]:
q = """WITH 
    tracks_sold AS
    (
        SELECT
            g.name genre,
            COUNT(il.quantity) tracks_sold,
            CAST(COUNT(il.quantity) as float)/(SELECT COUNT(*) FROM invoice_line) percentage_sold
        FROM invoice_line il
        INNER JOIN track t ON t.track_id = il.track_id
        INNER JOIN genre g ON g.genre_id = t.genre_id
        GROUP BY genre
        ORDER BY percentage_sold DESC
    ),
    tracks_available AS
    (
        SELECT
            g.name genre,
            COUNT(track_id) total_tracks,
            CAST(COUNT(track_id) as float)/(SELECT COUNT(*) FROM track) percentage
        FROM track t
        INNER JOIN genre g ON g.genre_id = t.genre_id
        GROUP BY g.name
    )

SELECT
    ta.genre,
    ts.tracks_sold,
    ts.percentage_sold,
    ta.total_tracks tracks_available,
    ta.percentage percentage_available
FROM tracks_available ta
LEFT JOIN tracks_sold ts ON ts.genre = ta.genre
ORDER BY percentage_sold DESC;"""

run_query(q)

,genre,tracks_sold,percentage_sold,tracks_available,percentage_available
0,Rock,2635.0,0.553921,1297,0.370254
1,Metal,619.0,0.130124,374,0.106766
2,Alternative & Punk,492.0,0.103427,332,0.094776
3,Latin,167.0,0.035106,579,0.165287
4,R&B/Soul,159.0,0.033424,61,0.017414
5,Blues,124.0,0.026067,81,0.023123
6,Jazz,121.0,0.025436,130,0.037111
7,Alternative,117.0,0.024595,40,0.011419
8,Easy Listening,74.0,0.015556,24,0.006851
9,Pop,63.0,0.013244,48,0.013703


## Do protected vs non-protected media types have an effect on popularity?


In [10]:
q = """SELECT
    mt.name media_type,
    COUNT(il.quantity) tracks_sold,
    CAST(COUNT(il.quantity) as float)/(SELECT COUNT(*) FROM invoice_line) percentage_sold
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
GROUP BY mt.media_type_id
ORDER BY percentage_sold DESC;"""

run_query(q)

,media_type,tracks_sold,percentage_sold
0,MPEG audio file,4259,0.895312
1,Protected AAC audio file,439,0.092285
2,Purchased AAC audio file,35,0.007358
3,AAC audio file,21,0.004415
4,Protected MPEG-4 video file,3,0.000631
